In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
import os

2023-09-04 17:14:50.780442: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 17:14:51.379975: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
sequences_data = [
    [1, 2, 3],
    [4, 5],
    [6, 7, 8, 9],
]

labels_data = [0, 1, 0]

sequences_ragged = tf.ragged.constant(sequences_data, dtype=tf.int32)
labels_ragged = tf.ragged.constant(labels_data, dtype=tf.int32)

dataset = tf.data.Dataset.from_tensor_slices((sequences_ragged, labels_ragged))

2023-09-04 17:14:52.783019: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43194 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [5]:
batched_dataset = dataset.batch(batch_size=2)

In [6]:
def rand_data(x):
    return (tf.fill([x], x), tf.constant([1], dtype=tf.int32))

In [98]:
A = (tf.data.Dataset
     .range(1,5, output_type=tf.int32)
     .map(rand_data))
A = A.padded_batch(2)

In [7]:
input_layer = tf.keras.layers.Input(shape=(None,), dtype=tf.int32)
embedding_layer = tf.keras.layers.Embedding(input_dim=10, output_dim=32, mask_zero=False)(input_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(embedding_layer)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam')

epochs = 5
batch_size = 2

model.fit(batched_dataset, epochs=epochs)

Epoch 1/5


2023-09-04 17:13:43.820807: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-04 17:13:43.821143: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0c70001950 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-04 17:13:43.821155: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2023-09-04 17:13:43.824558: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-04 17:13:44.910295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801
2023-09-04 17:13:44.996048: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 [==============================] - 3s 118ms/step - loss: 1.4710
Epoch 2/5
2/2 [==============================] - 0s 3ms/step - loss: 1.4691
Epoch 3/5
2/2 [==============================] - 0s 3ms/step - loss: 1.4671
Epoch 4/5
2/2 [==============================] - 0s 3ms/step - loss: 1.4651
Epoch 5/5
2/2 [==============================] - 0s 3ms/step - loss: 1.4631


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 32)          320       
                                                                 
 dense (Dense)               (None, None, 1)           33        
                                                                 
Total params: 353 (1.38 KB)
Trainable params: 353 (1.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
